In [ ]:
!pip install numpy matplotlib scikit-learn opencv-python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
img_path = '/content/mail imag.png'
img = cv2.imread(img_path)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_resized = cv2.resize(img_gray, (64, 64))
img_normalized = img_resized / 255.0

In [ ]:
img_expanded = np.expand_dims(img_normalized, axis=-1)
img_batch = np.expand_dims(img_expanded, axis=0)

In [ ]:
datagen = ImageDataGenerator(rotation_range=30, zoom_range=0.2, horizontal_flip=True)

augmented_images = []
for i, batch in enumerate(datagen.flow(img_batch, batch_size=1)):
    augmented_images.append(batch[0])
    if i >= 19:
        break


In [ ]:
X = np.array(augmented_images)
y = np.array([0] * 20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
cnn_model = Sequential()
cnn_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dense(1, activation='sigmoid'))

In [ ]:
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=4, batch_size=4, validation_split=0.2)
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Accuracy of CNN on augmented images: {cnn_accuracy * 100:.2f}%')

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Load the dataset
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
X = lfw_people.images
y = lfw_people.target

# Preprocess the data
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)  # Add channel dimension for grayscale
X = X / 255.0  # Normalize pixel values

# Convert labels to categorical
y = to_categorical(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {score[1]*100:.2f}%")

# Function to load and preprocess a custom image for testing
def preprocess_image(image_path, target_size):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
    img_resized = cv2.resize(img, target_size)  # Resize to match the CNN input
    img_resized_reshaped = img_resized.reshape(1, target_size[0], target_size[1], 1)  # Add batch and channel dimensions
    img_resized_reshaped = img_resized_reshaped / 255.0  # Normalize pixel values
    return img, img_resized_reshaped  # Return original and processed images

# Path to your custom test image
image_path = '/content/mail imag.png'  # Update this with your image path

# Preprocess the custom image
original_image, input_image = preprocess_image(image_path, (X_train.shape[1], X_train.shape[2]))

# Predict the class of the input image
prediction = model.predict(input_image)

# Get the class label
predicted_class = np.argmax(prediction)
predicted_label = lfw_people.target_names[predicted_class]

# Print the predicted class
print(f"Predicted class: {predicted_label}")

# Display the input image along with the predicted class
plt.imshow(original_image, cmap='gray')  # Display image in grayscale
plt.title(f"Predicted class: {predicted_label}")  # Set the title to show the predicted class
plt.axis('off')  # Hide the axis for better view
plt.show()

In [ ]:
# Import necessary libraries
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from google.colab import files
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

# Upload the image
uploaded = files.upload()

# Extract the filename from the uploaded files
image_path = list(uploaded.keys())[0]

# Function to extract features using VGG16 CNN
def extract_features_with_cnn(image_path):
    # Load the VGG16 model pre-trained on ImageNet, excluding the top layers (we're not interested in classification)
    base_model = VGG16(weights='imagenet', include_top=False)
    model = Model(inputs=base_model.inputs, outputs=base_model.get_layer('block5_pool').output)

    # Load and preprocess the image
    img = cv.imread(image_path)
    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img_resized = cv.resize(img_rgb, (224, 224))  # Resize image to 224x224 as required by VGG16
    img_array = np.expand_dims(img_resized, axis=0)  # Expand dimensions to fit batch format
    img_preprocessed = preprocess_input(img_array)  # Preprocess image

    # Extract features using the VGG16 model
    features = model.predict(img_preprocessed)

    # Reshape features to 2D (samples, features)
    features_reshaped = features.reshape(-1, features.shape[-1])

    return features_reshaped

# Function to perform K-means clustering and compare accuracy with and without PCA
def kmeans_with_and_without_pca(image_path, cluster_n):
    # Step 1: Feature extraction using CNN (VGG16 in this case)
    features = extract_features_with_cnn(image_path)

    # ---------- Without PCA ----------
    # Step 2: Perform K-means clustering on raw CNN features (without PCA)
    kmeans_no_pca = KMeans(n_clusters=cluster_n)
    labels_no_pca = kmeans_no_pca.fit_predict(features)

    # ---------- With PCA ----------
    # Step 4: Apply PCA to reduce dimensionality for clustering
    pca = PCA(n_components=2)
    features_pca = pca.fit_transform(features)

    # Step 5: Perform K-means clustering on PCA-transformed features
    kmeans_pca = KMeans(n_clusters=cluster_n)
    labels_pca = kmeans_pca.fit_predict(features_pca)

    # Step 6: Evaluate clustering with silhouette score (with PCA)
    silhouette_pca = silhouette_score(features_pca, labels_pca)
    print(f"Accuracy Score with PCA: {silhouette_pca:.4f}")

    # Step 7: Create a scatter plot for PCA
    plt.figure(figsize=(12, 6))
    scatter = plt.scatter(features_pca[:, 0], features_pca[:, 1], c=labels_pca, cmap='tab10', s=20)
    plt.colorbar(scatter, ticks=range(cluster_n), label='Cluster')
    plt.title('PCA of CNN-extracted Features (with K-means Clustering)')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.show()

# Number of clusters
cluster_n = 7

# Perform K-means clustering with and without PCA and compare silhouette scores
kmeans_with_and_without_pca(image_path, cluster_n)